In [1]:
# todo
# 16/3/23

he idea is to make some surface plots over the TD space to compare:
            resolution degradation for a fixed minimum AASR,
            swath portion over which RASR excedes a certain treshold,
            peak core snr
            NEXT what happens morphing the antenna size?

In [2]:
# From Michelangelo Villano's hands on lab
# To open separate plot-windows outside the browser uncomment one of the following two lines
%matplotlib qt
#get_ipython().run_line_magic('matplotlib','qt5')

# To open a Plot-window within notebook with zoom/edit control uncomment one of the following two lines
# %matplotlib notebook
# get_ipython().run_line_magic('matplotlib','notebook')

# options are 'osx', 'qt4', 'qt5', 'gtk3', 'wx', 'qt', 'gtk', 'tk' , 'notebook' , 'inline'

In [3]:
## Dependencies
from libs.timing_diagram import *
from libs.design_functions import *
import numpy as np
import matplotlib.pyplot as plt
from libs.spherical_earth_geometry_radar import *
from libs.radartools.farField import UniformAperture
from libs.ambiguity_functions import *
from unambiguous_imaging_design_analysis import *

_*Problem : how to sample the space:*_

**possibility 1** we restrict the sampling space to solutions which align at the far range (transmit event excluded) with the next nadir return.
- few points have to be computed
- some dynamics might be hidden

**possibility 2** we set a minimum and maximum range, we discretize the PRF axis, and we find all possible ranges for each sample within the swath.
- resolution set by the user
- the samples might not coincide with the nadir return avoidance optimal (a visual selection can still be made in any case).

**2** is better overall

Also we can use the intersection point of the two minprf and opt prf curves as a reference point, but how?

In [4]:
## System Parameters

# constants
c = 299792458

# radar parameters
dutycycle = 0.25  # duty cycle
h = 500e3  # height
wavelength = c / 10e9  # wavelength

# satellite speed
vs = orbital_speed(h)

# antenna size
Wa = 0.3  # antenna width in meters
La = 2  # antenna length in meters

# nadir duration in fractions of PRI for visualization in timing diagram
# note, it makes sense to use this fractional quantity as the nadir duration, if unfocused (e.g. saturated receiver), will be proportional to the impulse on time.
nadir_duration = 2 * dutycycle


In [5]:
## base diagram
# PRF axis
PRI = 1 / 7050
prf = np.linspace(1 / PRI - 4000, 1 / PRI + 4000, 100)

# plot
fig, ax = plt.subplots(1)
time_diagram_plotter(ax, prf, dutycycle, h, nadir=False, integrationtime=False)
nadir_return_plotter(ax, prf, dutycycle, nadir_duration, h)
ax.set_xlabel('PRF [Hz]')
ax.set_ylabel(' Ground range [km]')
ax.set_xlim(1 / PRI - 1000, 1 / PRI + 1000)
ax.set_ylim(100, 300)

C:\Users\smen851\PycharmProjects\systemtradeoffs\libs\design_functions.py:42: RuntimeWarning: invalid value encountered in arccos
  beta = arccos(((re + h) ** 2 - re ** 2 + rs ** 2) / (2 * (re + h) * rs))
C:\Users\smen851\PycharmProjects\systemtradeoffs\libs\design_functions.py:43: RuntimeWarning: invalid value encountered in arccos
  alpha = arccos(((re + h) ** 2 + re ** 2 - rs ** 2) / (2 * (re + h) * re))


(100.0, 300.0)

In [6]:
## canonical constraints line plots
# 1 ground range axis in m
ground_range = np.linspace(0, 2000, 500) * 1000

prf_opt = optimal_prf_line(ground_range, Wa, wavelength, h)
# Add to the plot
ax.plot(prf_opt, ground_range / 1000, 'red')

prf_min = minimum_prf_line(ground_range, vs, La, h)
ax.plot(prf_min, ground_range / 1000, 'green')

C:\Users\smen851\PycharmProjects\systemtradeoffs\unambiguous_imaging_design_analysis.py:56: RuntimeWarning: divide by zero encountered in divide
  prf_opt = 1 / pri


In [7]:
# find the intersection point
tosolve = lambda rgx: optimal_prf_line(rgx, Wa, wavelength, h) - minimum_prf_line(rgx, vs, La, h)

rg_point = fsolve(tosolve, 200000)
prf_point = minimum_prf_line(rg_point, vs, La, h)

print(prf_point, rg_point)

ax.plot(prf_point, rg_point / 1000, 'x', color='blue')

[7058.91598277] [197427.8613883]


At this point we have the timing diagram, the minimum PRF line, the maximum range line, and the intersetion point representing the ideal design solution.

The intersection point will be the center sample for the analysis; the extent is set empirically.

# Phase0: defining the meshgrid for calculation

In [8]:
# number of PRF samples
x_samples = 3
# number of ground range samples
y_samples = 4

prf_axis = np.linspace(prf_point - 1e3, prf_point + 1e3, x_samples)
prf_mesh = np.repeat(prf_axis, y_samples).reshape(x_samples, y_samples)
#print(prf_mesh)

# ground range mesh coordinates initialization
rg_mesh = np.zeros_like(prf_mesh)
for ii in range(x_samples):
    prf_point = prf_axis[ii]
    # the center range sample shall be on the red curve
    rg_opt = lambda rg: optimal_prf_line(rg, Wa, wavelength, h) - prf_point
    rg_point = fsolve(rg_opt, rg_point)  # todo red curve
    # 1 convert rg to slant range
    rs = range_ground_to_slant(rg_point, h)
    # 2 find the next pulse order
    n = np.ceil((2 * rs / c) * prf_point)
    # 3 find the nadir order
    n_nadir = np.ceil((2 * h / c) * prf_point)
    # 4 generate the range axis
    if n - n_nadir <= y_samples / 2:  # to maintain a side looking geometry and the same number of samples
        rs_axis = (np.arange(n_nadir, n_nadir + y_samples) + 0.5) * c / (2 * prf_point)
        rg_axis, tt = range_slant_to_ground(rs_axis, h)
    else:
        rs_axis = (np.arange(n - int(y_samples / 2) - 1, n - int(y_samples / 2) - 1 + y_samples) + 0.5) * c / (
                2 * prf_point)
        rg_axis, tt = range_slant_to_ground(rs_axis, h)
    rg_mesh[ii] = rg_axis

# it seems to work. tested with different values of x samples and y samples

In [9]:
print(prf_mesh)
print(rg_mesh)

[[6058.91598277 6058.91598277 6058.91598277 6058.91598277]
 [7058.91598277 7058.91598277 7058.91598277 7058.91598277]
 [8058.91598277 8058.91598277 8058.91598277 8058.91598277]]
[[174726.22012331 235593.7827447  285687.91255835 329953.14321422]
 [138438.15853905 200183.13214924 248635.63701629 290524.57559297]
 [103861.84345    169566.42887375 217626.48522901 258094.88507262]]


In [14]:
rg_mesh.shape

(3, 4)

In [10]:
# visualization
# plotting the points in the already open timing diagram.

X, Y = prf_mesh, rg_mesh

for ii in range(x_samples * y_samples):
    ax.plot(X.flatten()[ii], Y.flatten()[ii] / 1000, 'x', color='green')

# adjust the plot scale
ax.set_xlim(X.min() - 3e2, X.max() + 3e2)
ax.set_ylim(Y.min() / 1e3 - 30, Y.max() / 1e3 + 30);

# Phase1: Design optimization for every point of the grid

In [11]:
# maximum acceptable AASR level
AASR_max = -20  #dB
# maximum acceptable RASR level
RASR_max = -20  #dB

In [12]:
##  Initialization of the geometry model and aperture model

# Radar Geometry object
radar_geo = RadarGeometry()
radar_geo.set_rotation(30 * np.pi / 180, 0, 0)
radar_geo.set_initial_position(0, 0, h)
radar_geo.set_speed(vs)
# no need to set the radar rotation as this will be done inside the analysys function

# Uniform aperture antenna initialization
uniap = UniformAperture(La, Wa, c / wavelength)

In [15]:
# optimize the design for every point in the grid and store the results in a list
# also directly store the results of interest in separate lists # todo change to meshgrid logic
results_list = []
x_coordinate = []  # prf coordinate
y_coordinate = []  # optimized ground range coordinate (compressed image ground swath central point)
undersampling_list = []  # optimized undersampling ratio to limit AASR
rasr_free_swath_list = []  # portion of the ground range free from ambiguities
core_snr_list = []
for ii in range(x_samples):
    for jj in range(y_samples):
        dictionary = umambiguous_mode_analysis(radar_geo,
                                               uniap, rg_mesh[ii, jj],
                                               prf_mesh[ii, jj], AASR_max, RASR_max,
                                               vs, wavelength, dutycycle,
                                               aasr_points=6, rasr_points=27)
        # saving all the analysis results
        results_list.append(dictionary)
        # saving individual relevant figures from the analysis
        x_coordinate.append(np.average(dictionary['prf']))
        y_coordinate.append(
            np.average(dictionary['compressed-ground-swath']))  # this is the mid-point of the usable ground swath
        undersampling_list.append(dictionary['minimum-undersampling'])
        rasr_free_swath_list.append(dictionary['usable-rasr-swath'])
        core_snr_list.append(dictionary['core-snr-linear'])

optimized looking angle: 19.23830579294688


100%|██████████| 6/6 [00:11<00:00,  1.95s/it]


optimized looking angle: 25.14454768539982


100%|██████████| 6/6 [00:14<00:00,  2.36s/it]


optimized looking angle: 29.550811022428967


...

100%|██████████| 6/6 [00:10<00:00,  1.81s/it]


optimized looking angle: 23.49053548097997


100%|██████████| 6/6 [00:10<00:00,  1.76s/it]


optimized looking angle: 27.190775581897856


100%|██████████| 6/6 [00:10<00:00,  1.75s/it]


Note return dictionary of unambiguous_mode_analysis
    return_dict = {
        'ground-swath': swath_rg,
        'prf': swath_prf,
        'compressed-ground-swath': swath_rg_compressed,
        'compressed-slant-swath': swath_rs_compressed,
        'compressed-incidence-swath': swath_eta_compressed,
        'rasr-ground-range-axis': ground_range_axis,
        'rasr-slant-range-axis': slant_range_axis,
        'rasr-incidence-axis': incidence_axis,
        'rasr-linear': rasr,
        'rasr-doppler-bandwidth': Bd,
        'aasr-undersampling-axis': doppler_undersampling_ratio,
        'aasr-linear': aasr,
        'core-snr-linear': core_snr,
        'corrected-azimuth-resolution': azres,
        'minimum-undersampling': undersampling,
        'usable-rasr-swath': swath_rasr,
        'usable-swath-ranges': ranges
    }

In [20]:
# test
print(x_coordinate)

[6058.915982765739, 6058.915982765739, 6058.915982765739, 6058.915982765739, 7058.915982765739, 7058.915982765739, 7058.915982765739, 7058.915982765739, 8058.915982765739, 8058.915982765739, 8058.915982765739, 8058.915982765739]


# Phase1.5: results saver and loader for asynchronous operation
i.e. a cell to pickle the list and a cell to unpickle the list.


In [21]:
#todo

In [19]:
#todo

# Phase2: grid visualization

In [18]:
# plotting the points in the already open timing diagram.


for ii in range(len(x_coordinate)):
    ax.plot(x_coordinate[ii], y_coordinate[ii] / 1000, 'x', color='black')

# adjust the plot scale
ax.set_xlim(min(x_coordinate) - 3e2, max(x_coordinate) + 3e2)
ax.set_ylim(min(y_coordinate) / 1e3 - 30, max(y_coordinate) / 1e3 + 30)


(71.67003829538226, 359.838596616081)

## Phase2.5: colorbar visualization tool for design solutions
vertical lines corresponding to the usable (RASR free)swath
color of lines corresponding to resolution or core snr


In [ ]:
# prototype
def design_point_visualizer(axis, dictionary, scalar_element, colorbar, scalar_min, scalar_max, h=500e3, re=6371e3):
    """

    :param axis: matplotlib axis for the plot
    :param dictionary: dictionary from analysis
    :param scalar_element: a string containing the dictionary key pointing to a scalar value
    :param colorbar: colorbar
    :param scalar_min: minimum scalar value for the colorbar
    :param scalar_max: maximum scalar value for the colorbar
    :return: stocazzo
    """
    #1 find holes in the usable swath ranges, note the swath ranges are linearly spaced over the incidence angle axis
    ranges = dictionary['usable-swath-ranges']
    slant_ranges = range_ground_to_slant(ranges, h, re)
    ranges1, incidences = range_slant_to_ground(slant_ranges, h, re)
    deltas = incidences[1:-1] - incidences[0:-2]  # so far so good
    delta = min(deltas)
    holes = np.argwhere(deltas > 1.001 * delta)

    pass

In [34]:
# test
# insert a fake hole

dictionary = results_list[0]
ranges = dictionary['usable-swath-ranges']
slant_ranges = range_ground_to_slant(ranges, h)
ranges1, incidences = range_slant_to_ground(slant_ranges, h)
# test
print(max(ranges1-ranges))
deltas = incidences[1:-1] - incidences[0:-2]
delta = min(deltas)
holes = np.argwhere(deltas > 1.001 * delta)
print(holes)

[2.81143002e-08]
[]


In [37]:
print(holes.size)

0


# Phase3: Azimuth resolution, or undersampling ratio visualization